# Command-Line Interface

> Command-line tools for barcode extraction and processing

## Introduction
This notebook implements the command-line interface (CLI) for BarcodeSeqKit, allowing users to easily extract barcodes from BAM and FASTQ files without writing Python code.

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import re
import gzip
import logging
import shutil
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any, Counter
from pathlib import Path
import pysam
import tempfile
import pysam
import numpy as np
from pysam import AlignmentFile, AlignedSegment
import subprocess
import argparse
import sys

from tqdm.auto import tqdm
from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeMatch, 
    OrientationType,
    BarcodeExtractor, 
    ExtractorConfig,
    ExtractionStatistics,
    ExtractorFactory,
    BarcodeLocationType,
    FileFormat
)

from BarcodeSeqKit.sequence_utils import classify_read_by_first_match

# Main CLI Setup
Let's define the main entry point for the command-line interface:

Now let's define the arguments for the extract command:

In [ ]:
#| export
def add_extract_arguments(parser: argparse.ArgumentParser):
    """Add arguments for the extract command.
    
    Args:
        parser: ArgumentParser to add arguments to
    """
    # Input files (required)
    input_group = parser.add_argument_group('Input options')
    
    # Input can be specified in different ways
    input_mutex = input_group.add_mutually_exclusive_group(required=True)
    
    input_mutex.add_argument(
        '--bam',
        help='Input BAM file'
    )
    
    input_mutex.add_argument(
        '--fastq1',
        help='Input FASTQ file (read 1 for paired-end data)'
    )
    
    input_mutex.add_argument(
        '--fastq-dir',
        help='Directory containing paired FASTQ files'
    )
    
    # When specifying fastq1, fastq2 is optional
    input_group.add_argument(
        '--fastq2',
        help='Input FASTQ file (read 2 for paired-end data, required if --fastq1 is used)'
    )
    
    # Barcode configuration (required)
    barcode_group = parser.add_argument_group('Barcode options')
    
    barcode_mutex = barcode_group.add_mutually_exclusive_group(required=True)
    
    barcode_mutex.add_argument(
        '--barcode-config',
        help='YAML file containing barcode configurations'
    )
    
    barcode_mutex.add_argument(
        '--barcode',
        help='Single barcode sequence (no specific location, will generate barcode_orientFR and barcode_orientRC outputs)'
    )
    
    barcode_mutex.add_argument(
        '--barcode5',
        help="5' barcode sequence (will generate barcode5_orientFR and barcode5_orientRC outputs)"
    )
    
    barcode_group.add_argument(
        '--barcode3',
        help="3' barcode sequence (will generate barcode3_orientFR and barcode3_orientRC outputs, can be used with --barcode5)"
    )
    
    barcode_group.add_argument(
        '--max-mismatches',
        type=int,
        default=0,
        help='Maximum number of mismatches allowed in barcode detection (default: 0)'
    )
    
    # Output options
    output_group = parser.add_argument_group('Output options')
    
    output_group.add_argument(
        '--output-prefix',
        required=True,
        help='Prefix for output files'
    )
    
    output_group.add_argument(
        '--output-dir',
        default='.',
        help='Directory for output files (default: current directory)'
    )
    
    output_group.add_argument(
        '--discard-unmatched',
        action='store_true',
        help='Discard reads without barcode matches'
    )
    
    # Processing options
    processing_group = parser.add_argument_group('Processing options')
    
    processing_group.add_argument(
        '--search-both-reads',
        action='store_true',
        help='Search for barcodes in both reads of paired-end data (FASTQ only)'
    )
    
    processing_group.add_argument(
        '--search-mate',
        action='store_true',
        help='Search for barcodes in mate reads (BAM only)'
    )
    
    processing_group.add_argument(
        '--no-compress',
        action='store_true',
        help='Do not compress output FASTQ files (FASTQ only)'
    )
    
    # Logging options
    logging_group = parser.add_argument_group('Logging options')
    
    logging_group.add_argument(
        '--verbose',
        action='store_true',
        help='Enable verbose logging'
    )
    
    logging_group.add_argument(
        '--log-file',
        help='Log file path'
    )
    return parser



In [ ]:
#| export
def run_cli(args: Optional[List[str]] = None) -> int:
    """Handle the extract command.
    
    Args:
        args: Command-line arguments
        
    Returns:
        Exit code (0 for success, non-zero for error)
    """
    parser = argparse.ArgumentParser(
        description="BarcodeSeqKit: A toolkit for barcode extraction from sequencing data",
        formatter_class=argparse.RawDescriptionHelpFormatter
    )   
    
    parser = add_extract_arguments(parser)
    
    try:
        parsed_args = parser.parse_args(args)
        
        # Set up logging - prevent duplicate handlers
        logger = logging.getLogger('BarcodeSeqKit')
        logger.handlers = []  # Clear any existing handlers
        
        log_level = logging.DEBUG if parsed_args.verbose else logging.INFO
        logger.setLevel(log_level)
        
        # Remove any root logger handlers to prevent duplicate messages
        root_logger = logging.getLogger()
        root_logger.handlers = []
        
        # Add console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(log_level)
        console_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(console_format)
        logger.addHandler(console_handler)
        
        # Add file handler if requested
        if parsed_args.log_file:
            file_handler = logging.FileHandler(parsed_args.log_file)
            file_handler.setLevel(log_level)
            file_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(file_format)
            logger.addHandler(file_handler)
        
        # Prevent propagation to root logger
        logger.propagate = False
        
        # Create output directory if it doesn't exist
        os.makedirs(parsed_args.output_dir, exist_ok=True)
        
        # Determine the input file format and paths
        input_files = []
        
        if parsed_args.bam:
            input_files = [parsed_args.bam]
            logger.info(f"Input BAM file: {parsed_args.bam}")
        elif parsed_args.fastq1:
            if parsed_args.fastq2:
                input_files = [parsed_args.fastq1, parsed_args.fastq2]
                logger.info(f"Input FASTQ files: {parsed_args.fastq1}, {parsed_args.fastq2}")
            else:
                input_files = [parsed_args.fastq1]
                logger.info(f"Input FASTQ file: {parsed_args.fastq1}")
        elif parsed_args.fastq_dir:
            input_files = [parsed_args.fastq_dir]
            logger.info(f"Input FASTQ directory: {parsed_args.fastq_dir}")
        
        # Load or create barcode configurations
        barcodes = []
        
        if parsed_args.barcode_config:
            # Load from YAML file
            import yaml
            with open(parsed_args.barcode_config, 'r') as f:
                config_data = yaml.safe_load(f)
            
            if 'barcodes' in config_data:
                for barcode_data in config_data['barcodes']:
                    location = BarcodeLocationType(barcode_data.get('location', 'UNK'))
                    barcodes.append(BarcodeConfig(
                        sequence=barcode_data['sequence'],
                        location=location,
                        name=barcode_data.get('name'),
                        description=barcode_data.get('description')
                    ))
            else:
                logger.error("Invalid barcode configuration file")
                return 1
        else:
            # Create from command-line arguments
            if parsed_args.barcode:
                # Single barcode mode (no specific location)
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode,
                    location=BarcodeLocationType.UNKNOWN,
                    name="generic",
                    description="Generic barcode from command line"
                ))
                logger.info(f"Using single barcode mode with sequence: {parsed_args.barcode}")
            elif parsed_args.barcode5:
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode5,
                    location=BarcodeLocationType.FIVE_PRIME,
                    name="5",
                    description="5' barcode from command line"
                ))
                logger.info(f"Using 5' barcode with sequence: {parsed_args.barcode5}")
                
                if parsed_args.barcode3:
                    barcodes.append(BarcodeConfig(
                        sequence=parsed_args.barcode3,
                        location=BarcodeLocationType.THREE_PRIME,
                        name="3",
                        description="3' barcode from command line"
                    ))
                    logger.info(f"Using 3' barcode with sequence: {parsed_args.barcode3}")
            elif parsed_args.barcode3:
                # If only barcode3 is provided without barcode5, treat as single barcode
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode3,
                    location=BarcodeLocationType.UNKNOWN,
                    name="generic",
                    description="Generic barcode from command line (provided as barcode3)"
                ))
                logger.info(f"Using single barcode mode with sequence: {parsed_args.barcode3} (from barcode3)")
        
        # Validate that we have at least one barcode
        if not barcodes:
            logger.error("No barcode configurations provided")
            return 1
        
        # Determine if we're in single barcode mode
        single_barcode_mode = len(barcodes) == 1 or all(b.location.value == "UNK" for b in barcodes)
        output_description = "barcode_orientFR/RC" if single_barcode_mode else "barcode5_orientFR/RC/barcode3_orientFR/RC"
        logger.info(f"Will create output files with pattern: {parsed_args.output_prefix}_{output_description}.bam")
        
        # Create extractor configuration
        config = ExtractorConfig(
            barcodes=barcodes,
            output_prefix=parsed_args.output_prefix,
            output_dir=parsed_args.output_dir,
            merge_orientations=False,  # Always set to False with new behavior
            keep_unmatched=not parsed_args.discard_unmatched,
            verbose=parsed_args.verbose,
            log_file=parsed_args.log_file
        )
        
        # Save the configuration for reproducibility
        config_path = os.path.join(parsed_args.output_dir, f"{parsed_args.output_prefix}_config.yaml")
        config.save_yaml(config_path)
        logger.info(f"Saved configuration to {config_path}")
        
        # Create and run the appropriate extractor
        extractor = ExtractorFactory.create_extractor(config, input_files)
        
        # Set additional extractor options based on file format
        if FileFormat.detect_format(input_files[0]) == FileFormat.BAM:
            if hasattr(extractor, 'search_mate'):
                extractor.search_mate = parsed_args.search_mate
            if hasattr(extractor, 'max_mismatches'):
                extractor.max_mismatches = parsed_args.max_mismatches
        else:  # FASTQ
            if hasattr(extractor, 'search_both_reads'):
                extractor.search_both_reads = parsed_args.search_both_reads
            if hasattr(extractor, 'compress_output'):
                extractor.compress_output = not parsed_args.no_compress
            if hasattr(extractor, 'max_mismatches'):
                extractor.max_mismatches = parsed_args.max_mismatches
        
        # Run the extraction
        logger.info("Starting barcode extraction")
        stats = extractor.extract()
        
        # Print summary statistics
        logger.info("Extraction complete")
        logger.info(f"Total reads: {stats.total_reads}")
        logger.info(f"Barcode matches: {stats.total_barcode_matches}")
        logger.info(f"No barcode: {stats.no_barcode_count}")
        
        if stats.total_reads > 0:
            match_rate = (stats.total_barcode_matches / stats.total_reads) * 100
            logger.info(f"Match rate: {match_rate:.2f}%")
        
        # Log barcode-specific statistics
        for barcode_name, count in stats.matches_by_barcode.items():
            logger.info(f"Barcode {barcode_name}: {count} matches")
        
        # Log orientation-specific statistics
        for orientation, count in stats.matches_by_orientation.items():
            logger.info(f"Orientation {orientation}: {count} matches")
        
        # Log category-specific statistics
        if hasattr(stats, 'matches_by_category') and stats.matches_by_category:
            logger.info("Match statistics by category:")
            for category, count in stats.matches_by_category.items():
                logger.info(f"  {category}: {count} matches")
            
        return 0
        
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return 1

In [ ]:
#| export
def main() -> int:
    """Main entry point for command-line execution."""
    # Only actually call sys.exit() when running as a script, not in tests or notebooks
    is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
    # Use test arguments when in test/notebook environment
    if is_notebook_or_test:
        # For testing, you can add test code here
        return 0
    else:
        return run_cli()

In [ ]:
#| eval: false
#| export
if __name__ == "__main__":
    # For safety, wrap in try/except to catch any SystemExit
    try:
        exit_code = main()
        is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
        if is_notebook_or_test:
            print(f"\nCLI completed with exit code: {exit_code}")
    except SystemExit as e:
        # Just in case, catch any SystemExit and print instead
        is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
        if is_notebook_or_test:
            print(f"\nSystemExit caught with code: {e.code}")
        else:
            raise


CLI completed with exit code: 0


In [ ]:
#'GGCTAACTGAGGCCGGCCGAAGGCGATGATCACAACACTTAACGAACAGTGGAACGGTTTGAAGGTGAAAGGGGGT'

In [ ]:
#'CTGACTCCTTAAGGGCC' 'TAACTGAGGCCGGC'
#!barcodeseqkit --bam ../tests/test.bam --barcode5 CTGACTCCTTAAGGGCC --barcode3 TAACTGAGGCCGGC --output-prefix test_out_2 --output-dir ../tests/test_out_2

In [ ]:
#!barcodeseqkit --bam ../tests/test.bam --barcode CTGACTCCTTAAGGGCC --output-prefix test_out_1 --output-dir ../tests/test_out_1

In [ ]:
#!samtools view -b /Users/MTinti/git_projects/test_bam/BSF_04_sorted.bam \
#Chr10_core_Tb427v10:3403063-3406436 > /Users/MTinti/git_projects/test_bam/test.bam

In [ ]:
#ls /Users/MTinti/git_projects/

In [ ]:
#ls /Users/MTinti/git_projects/BarcodeSeqKit/tests

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()